In [3]:
def get_marks(pdf_paths):
    
    for pdf_path in pdf_paths:
        from PIL import Image
        import cv2
        import numpy as np
        import pandas as pd 
        import tensorflow as tf
        from paddleocr import PaddleOCR, draw_ocr
        import glob, sys, fitz
        import layoutparser as lp


        # To get better resolution
        zoom_x = 2.0  # horizontal zoom
        zoom_y = 2.0  # vertical zoom
        mat = fitz.Matrix(zoom_x, zoom_y)  # zoom factor 2 in each dimension

        doc = fitz.open(pdf_path)  # open document
        for page in doc:
            pix = page.get_pixmap(matrix=mat)  # render page to an image
            pix.save("saved.png")  # store image as a PNG

        image = cv2.imread("saved.png")

        image = image[..., ::-1]

        # load model
        model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                        threshold=0.5,
                                        label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
                                        enforce_cpu=False,
                                        enable_mkldnn=True)#math kernel library
        layout = model.detect(image)
        x_1=0
        y_1=0
        x_2=0
        y_2=0

        for l in layout:
          print(l)
          if l.type == 'Table':
            x_1 = int(l.block.x_1)
            print(l.block.x_1)
            y_1 = int(l.block.y_1)
            x_2 = int(l.block.x_2)
            y_2 = int(l.block.y_2)

            break

        im = cv2.imread('saved.png')
        cv2.imwrite('105_page-0001/ext_im.jpg', im)

        ocr = PaddleOCR(lang='en')
        image_path = '105_page-0001/ext_im.jpg'
        image_cv = cv2.imread(image_path)
        image_height = image_cv.shape[0]
        image_width = image_cv.shape[1]
        output = ocr.ocr(image_path)

        boxes = [line[0] for line in output]
        texts = [line[1][0] for line in output]
        probabilities = [line[1][1] for line in output]

        image_boxes = image_cv.copy()

        for box,text in zip(boxes,texts):
          cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
          cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

        #cv2.imwrite('105_page-0001/detections.jpg', image_boxes)

        im = image_cv.copy()

        horiz_boxes = []
        vert_boxes = []

        for box in boxes:
          x_h, x_v = 0,int(box[0][0])
          y_h, y_v = int(box[0][1]),0
          width_h,width_v = image_width, int(box[2][0]-box[0][0])
          height_h,height_v = int(box[2][1]-box[0][1]),image_height

          horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
          vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

          cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
          cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)

        #cv2.imwrite('105_page-0001/horiz_vert.jpg',im)

        horiz_out = tf.image.non_max_suppression(
            horiz_boxes,
            probabilities,
            max_output_size = 1000,
            iou_threshold=0.1,
            score_threshold=float('-inf'),
            name=None
        )

        horiz_lines = np.sort(np.array(horiz_out))

        im_nms = image_cv.copy()

        for val in horiz_lines:
          cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)
        #cv2.imwrite('105_page-0001/im_nms.jpg',im_nms)

        vert_out = tf.image.non_max_suppression(
            vert_boxes,
            probabilities,
            max_output_size = 1000,
            iou_threshold=0.1,
            score_threshold=float('-inf'),
            name=None
        )

        vert_lines = np.sort(np.array(vert_out))

        for val in vert_lines:
          cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)
        #cv2.imwrite('105_page-0001/im_nms.jpg',im_nms)

        out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]

        unordered_boxes = []

        for i in vert_lines:
          unordered_boxes.append(vert_boxes[i][0])

        ordered_boxes = np.argsort(unordered_boxes)

        def intersection(box_1, box_2):
          return [box_2[0], box_1[1],box_2[2], box_1[3]]

        def iou(box_1, box_2):

          x_1 = max(box_1[0], box_2[0])
          y_1 = max(box_1[1], box_2[1])
          x_2 = min(box_1[2], box_2[2])
          y_2 = min(box_1[3], box_2[3])

          inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
          if inter == 0:
              return 0

          box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
          box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

          return inter / float(box_1_area + box_2_area - inter)

        for i in range(len(horiz_lines)):
          for j in range(len(vert_lines)):
            resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

            for b in range(len(boxes)):
              the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
              if(iou(resultant,the_box)>0.1):
                out_array[i][j] = texts[b]
        out_array=np.array(out_array)

        pd.DataFrame(out_array).to_csv('105_page-0001/sample.csv')
        import pandas as pd
        df = pd.read_csv("105_page-0001/sample.csv")
        df = df.drop("Unnamed: 0",axis=1)
        x=df.shape
        math = 0
        science = 0
        eng = 0
        flag1 = False
        flag2 = False
        flag3 = False

        for j in range(x[1]-2):
          for i in range(x[0]):
            col = str(j)
            string1 = str(df.iloc[i][col])
            if "12" in string1:
              if flag1 == False:  
                math = i
                flag1 = True

            elif "11" in string1:
              if flag2 == False:
                science = i
                flag2 = True

            elif "16" in string1:
              if flag3 == False:
                eng = i
                flag3 = True

            elif flag1 == True and flag2 == True and flag3 == True:
              break

            else:
              pass


        math_row = list(df.iloc[math])
        science_row = list(df.iloc[science])
        eng_row = list(df.iloc[eng])

        marks_math=[]
        marks_sci=[]
        marks_eng=[]

        for i in math_row:
          try:
            marks_math.append(int(i))
          except:
            pass
        for i in science_row:
          try:
            marks_sci.append(int(i))
          except:
            pass
        for i in eng_row:
          try:
            marks_eng.append(int(i))
          except:
            pass


        from functools import reduce

        maths=reduce(max,marks_math)
        sciences=reduce(max,marks_sci)
        english=reduce(max,marks_eng)

        total_Maximum_marks = 300
        total_Obtained_marks = maths + sciences + english

        import mysql.connector

        mydb = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = "root",database='Marks_extractor')
        cursor = mydb.cursor()
        sql = "INSERT INTO marksheet (maths,science,english,total,obtained) VALUES (%s, %s, %s, %s, %s)"
        val = (maths,sciences,english,total_Maximum_marks,total_Obtained_marks)

        cursor.execute(sql, val)
        mydb.commit()
    result()

In [1]:

import tkinter as tk
from tkinter import filedialog
from tkinter import *
from turtle import width
from PIL import ImageTk, Image
import numpy as np
import cv2
from matplotlib import pyplot 
import matplotlib.pylab as plt
from numpy import asarray
import time
import os
import imageio
import tkinter as tk, threading
import PIL.Image


# Importing tkinter to make gui in python 
from tkinter import*
  
from tkPDFViewer import tkPDFViewer as pdf 
global root
# Initializing tk 
root = Tk() 
  
# Set the width and height of our root window. 
root.geometry("1920x1080") 

left_frame = Frame(root, padx=15, pady=15)
left_frame.grid(row=0, column=0)
left_frame.place(x=1, y=5)
upper_left_frame = Frame(left_frame)
upper_left_frame.grid(row=0, column=0)
logo = ImageTk.PhotoImage(PIL.Image.open("logo-acpdc.png").resize((100, 100)))
in_frame = Label(upper_left_frame, image=logo)
in_frame.pack()

top_frame = LabelFrame(root, padx=15, pady=15, width=1000, height=100)
top_frame.grid(row=0, column=1)
top_frame.place(x=600, y=15)

global bottom_frame

image_btn_frame = Frame(top_frame, padx=15, pady=15)
image_btn_frame.grid(row=0, column=1)
# video_btn_frame = Frame(top_frame, padx=15, pady=15)
# video_btn_frame.grid(row=0, column=1)
image_btn = Button(image_btn_frame, text="Upload PDF", command=lambda: pdf1())
image_btn.configure(background = "#364156", foreground='white', font =('arial',15,'bold'))
image_btn.pack()

def pdf1():
    bottom_frame = LabelFrame(root, padx=15, pady=15, width=670, height=500)
    bottom_frame.grid(row=1, column=1)
    bottom_frame.place(x=350, y=150)
    file_path = list(filedialog.askopenfilenames())
    for f in file_path:
        v1 = pdf.ShowPdf() 
        # Adding pdf location and width and height. 
        v2 = v1.pdf_view(bottom_frame, 
                        pdf_location = f,  
                        width = 75, height = 25)

    # Placing Pdf in my gui. 
    v2.pack() 
    Extract_btn = Button(bottom_frame, text="Extract Marks", command=lambda: get_marks(file_path) )
    Extract_btn.configure(background = "#364156", foreground='white', font =('arial',15,'bold'))
    Extract_btn.pack()

def result():
    win  = Toplevel(root)
    win.geometry('1920x1080')

    result_frame = LabelFrame(win, padx=15, pady=15, width=1000, height=100)
    result_frame.grid(row=0, column=1)
    result_frame.place(x=650, y=300)

    text = Label(win, text="Result saved Successfully")
    text.place(x=520,y=220)
    text.config(font=('Helvetica bold',40))

    btn2 = Button(result_frame, text="OK", command=win.destroy)
    btn2.configure(background = "#364156", foreground='white', font =('arial',15,'bold'))
    btn2.pack()

root.mainloop()